<a href="https://colab.research.google.com/github/Nerdward/Ml-Projects/blob/main/cifar10cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/Ml-Projects/cifar10/

/content/drive/MyDrive/Ml-Projects/cifar10


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_epochs= 50
batch_size = 64
learning_rate = 0.001

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0,0,0),(1,1,1))]
)

In [ ]:
train_dataset = datasets.CIFAR10(root='./data',train=True,download=True,
                                 transform=transform)
test_dataset = datasets.CIFAR10(root='./data',train=False,download=True,
                                 transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,
                                           shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, 5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(32, 64, 3)
    self.fc1 = nn.Linear(64*6*6, 128)
    self.drop = nn.Dropout(0.5)
    self.fc2 = nn.Linear(128,10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 64*6*6)
    x = self.drop(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

In [ ]:
def train(model, device, train_loader, optimizer, loss_fn, epoch):
  model.train()
  size = len(train_loader.dataset)
  for batch, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    output = model(data)
    loss = loss_fn(output, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch*len(data)
      print(f'loss: {loss:>7f} [{current}/{size}]')

In [ ]:
def test(model, device, test_loader, loss_fn):
  model.eval()
  size = len(test_loader.dataset)
  batch = len(test_loader)
  test_loss, correct = 0,0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += loss_fn(output, target)
      correct += (output.argmax(1) == target).type(torch.float).sum().item()

  test_loss /= batch
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
model = ConvNet().to(device)
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(num_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model,device, train_loader,opt, loss, t)
    test(model, device, test_loader,loss)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301749 [0/50000]
loss: 1.883309 [6400/50000]
loss: 1.662535 [12800/50000]
loss: 1.656819 [19200/50000]
loss: 1.630599 [25600/50000]
loss: 1.308736 [32000/50000]
loss: 1.673162 [38400/50000]
loss: 1.383196 [44800/50000]
Test Error: 
 Accuracy: 49.1%, Avg loss: 1.417417 

Epoch 2
-------------------------------
loss: 1.621932 [0/50000]
loss: 1.400966 [6400/50000]
loss: 1.460614 [12800/50000]
loss: 1.347626 [19200/50000]
loss: 1.237794 [25600/50000]
loss: 1.433347 [32000/50000]
loss: 1.363052 [38400/50000]
loss: 1.550015 [44800/50000]
Test Error: 
 Accuracy: 54.9%, Avg loss: 1.261419 

Epoch 3
-------------------------------
loss: 1.367412 [0/50000]
loss: 1.311779 [6400/50000]
loss: 1.163010 [12800/50000]
loss: 1.451280 [19200/50000]
loss: 1.241017 [25600/50000]
loss: 1.284403 [32000/50000]
loss: 1.095576 [38400/50000]
loss: 1.314041 [44800/50000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.142439 

Epoch 4
-------------------------------
los

KeyboardInterrupt: ignored